# Calculating a Zero Beta Portfolio #

### First Approach: Five Stocks ###

In [61]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Plots
import matplotlib.pyplot as plt

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from functions import import_financial_data
from capm_toolkit import compute_daily_returns
from capm_toolkit import compute_excess_returns
from capm_toolkit import capm_regression

# Pretty Notation
from IPython.display import display, Math

In [52]:
# Get the important data for the Risk Free Rate

rfr = pd.read_csv(rf"..\additional_data\rfr.csv")
rfr = rfr.set_index('Date')
rfr.index = pd.to_datetime(rfr.index, dayfirst=True)
rfr.dropna(inplace = True)

rfr

In [34]:
# Get the important data for the S&P500

sp500 = pd.read_csv(rf"..\additional_data\sp500.csv")
sp500 = sp500.set_index('Date')
sp500.index = pd.to_datetime(sp500.index)

In [35]:
# Get the important data for the S&P500

betas_df = pd.read_csv(rf"..\additional_data\capm_hbetas.csv")
betas_df = betas_df.set_index('date')
betas_df.index = pd.to_datetime(betas_df.index)

In [36]:
# Import Data

# Apple Data
df_1 = import_financial_data('AAPL')

# Amazon Data
df_2 = import_financial_data('AMZN')

# Meta Data
df_3 = import_financial_data('META')

# Microsoft Data
df_4 = import_financial_data('MSFT')

# Nvidia Data
df_5 = import_financial_data('NVDA')


In [37]:
# Let us calculate the Betas for some stocks

# Apple
apple_beta = betas_df['AAPL']

# Amazon
amazon_beta = betas_df['AMZN']

# META
meta_beta = betas_df['META']

# Microsoft
microsoft_beta = betas_df['MSFT']

# Nvidia
nvdia_beta = betas_df['NVDA']

In [38]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(apple_beta, label='Apple Beta', color='blue', alpha=0.7)
plt.plot(amazon_beta, label='Amazon Beta', color='red', alpha=0.7)
plt.plot(meta_beta, label='META Beta', color='green', alpha=0.7)
plt.plot(microsoft_beta, label='Microsoft Beta', color='orange', alpha=0.7)
plt.plot(nvdia_beta, label='NVDIA Beta', color='purple', alpha=0.7)
plt.axhline(y=1, color='black', linestyle='dashed')

# Config
plt.title('Beta Time Series')
plt.xlabel('Time')
plt.ylabel('Beta')
plt.legend()

# Show
plt.show()

In [39]:
# Let us find an insensible portfolio to the market

data_portfolio = pd.DataFrame()

data_portfolio['AAPL'] = df_1['adj_close']
data_portfolio['AMZN'] = df_2['adj_close']
data_portfolio['META'] = df_3['adj_close']
data_portfolio['MSFT'] = df_4['adj_close']
data_portfolio['NVDA'] = df_5['adj_close']

data_portfolio = data_portfolio.dropna()

data_portfolio = data_portfolio.loc['2016-01-01':]

data_portfolio

In [40]:
# Returns

df_returns = data_portfolio.pct_change(1).dropna()

df_returns

In [41]:
# Common Index

common_index = df_returns.index.intersection(betas_df.index)  # Common DATE
df_returns = df_returns.reindex(common_index)
betas_df = betas_df.reindex(common_index)

In [42]:
# We have to find the mean expected returns

expected_returns = df_returns.mean()

expected_returns

In [43]:
# We have to also find the standard deviations

volat = df_returns.dropna().std()

volat

In [44]:
# Also we have to find the covariance matrix

cov_matrix = df_returns.dropna().cov()

cov_matrix

In [45]:
# We need also to find the mean betas of the stocks

sample_beta_df = pd.DataFrame()

sample_beta_df['AAPL'] = apple_beta
sample_beta_df['AMZN'] = amazon_beta
sample_beta_df['META'] = meta_beta
sample_beta_df['MSFT'] = microsoft_beta
sample_beta_df['NVDA'] = nvdia_beta

mean_betas = sample_beta_df.mean()

mean_betas

In [46]:
# Which is the form of the portfolio weights if we want a zero-beta portfolio

display(Math(r"\omega=\left(\frac{D}{DC-E^2}\right)\Sigma^{-1}\iota+\left(\frac{E}{DC-E^2}\right)\Sigma^{-1}\beta"))
display(Math(r"C = \iota^⊤\Sigma^{-1}\iota"))
display(Math(r"D = \beta^⊤\Sigma^{-1}\beta"))
display(Math(r"E = \beta^⊤\Sigma^{-1}\iota"))
display(Math(r"\Delta = DC-E^2"))

In [47]:
# So let us obtain the components

beta = mean_betas.values.flatten().reshape(-1, 1)
Sigma = cov_matrix.values

lambda_ = 1e-6  # Tikhonov Regularization
Sigma_inv = np.linalg.inv(Sigma + lambda_ * np.eye(Sigma.shape[0]))

iota = np.ones((5, 1))

In [48]:
# And now obtain the coefficients of the Efficient Frontier

C = np.dot(np.dot(iota.T, Sigma_inv), iota)
D = np.dot(np.dot(beta.T, Sigma_inv), beta)
E = np.dot(np.dot(beta.T, Sigma_inv), iota)
Delta = (D*C - E*E)

print(f"This is C: {C[0][0]}")
print(f"This is D: {D[0][0]}")
print(f"This is E: {E[0][0]}")
print(f"This is Delta: {Delta[0][0]}")

In [49]:
# Now the weights

beta_weights = ((D/Delta)*(Sigma_inv @ iota)) - ((E/Delta)*(Sigma_inv @ beta))

beta_weights

In [50]:
# Create the portfolio

df_returns["zero_beta"] = df_returns.iloc[:, :5] @ beta_weights

df_returns

In [53]:
# Create the excess returns
df_returns['daily_risk_free_rate'] = compute_daily_returns(rfr['risk_free_rate'])
df_returns['portfolio_excess'] = df_returns['zero_beta'] - df_returns['daily_risk_free_rate']
df_returns['market_excess'] = compute_excess_returns(sp500['sp_500'], rfr['risk_free_rate'])

df_returns

In [54]:
# Calculate the beta with the weights

zero_beta_alt = mean_betas.transpose().dot(beta_weights)

print(zero_beta_alt.round(4))

In [55]:
def calculate_analytics(df_returns, risk_free_rate=0.0):
    # Trading Days in one Year
    ann_factor = 252  
    
    # Annualized Returns
    annualized_return = df_returns.mean() * ann_factor
    
    # Annualized Volatility
    annualized_std = df_returns.std() * np.sqrt(ann_factor)
    
    # Sharpe Ratio
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_std
    
    # Max Drawdown
    cumulative_returns = (1 + df_returns.div(100)).cumprod()
    rolling_max = cumulative_returns.cummax()
    drawdown = (cumulative_returns / rolling_max) - 1
    max_drawdown = drawdown.min()

    # VaR at 95%
    var_95 = df_returns.quantile(0.05)

    # Create DF
    summary_df = pd.DataFrame({
        "Annualized Returns": annualized_return,
        "Annualized Volatility": annualized_std,
        "Sharpe Ratio": sharpe_ratio,
        "Max Drawdown": max_drawdown,
        "VaR 95%": var_95
    })
    
    return summary_df

In [56]:
# Create a DataFrame

tickers = ["AAPL", "AMZN", "META", "MSFT", "NVDA", "zero_beta"]

df_portfolios = df_returns[tickers]

df_portfolios

In [57]:
# And the Table

analytics_table = calculate_analytics(df_portfolios)

analytics_table

In [58]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(df_returns['AAPL'].cumsum(), label='APPL Price', color='blue', alpha=0.7)
plt.plot(df_returns['AMZN'].cumsum(), label='AMZN Price', color='green', alpha=0.7)
plt.plot(df_returns['META'].cumsum(), label='META Price', color='red', alpha=0.7)
plt.plot(df_returns['MSFT'].cumsum(), label='MSFT Price', color='orange', alpha=0.7)
plt.plot(df_returns['NVDA'].cumsum(), label='NVDA Portfolio', color='purple', alpha=0.7)
plt.plot(df_returns['zero_beta'].cumsum(), label='Zero Beta Portfolio', color='black', alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time Index')
plt.ylabel('Cumulative Returns')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [59]:
# Create Figure
fig, ax1 = plt.subplots(dpi = 300)

# SP500 Returns
sp500['sp_500'].pct_change(1).mul(100).cumsum().plot(color = 'blue', ax = ax1)
ax1.set_xlabel('Date')
ax1.set_ylabel(
    'SP500 Returns', 
    color='blue'
    )

# Zero Beta Portfolio Returns
ax2 = ax1.twinx()

df_returns['zero_beta'].cumsum().plot(color = 'red', ax = ax2)
ax2.set_ylabel(
    'Zero Beta Portfolio Returns', 
    color='red'
    )

plt.show()

In [68]:
# Calculate the beta

window = len(df_returns)

#Model specification
results = capm_regression(
    df_returns['portfolio_excess'],
    df_returns['market_excess'],
    window,
    True
)
    
#here we check the summary
print(results.summary())       